In [10]:
import re
import string
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer



def retrieve_docs_and_clean():
  main_df = pd.read_csv(r'C:\Users\ammar\OneDrive\Desktop\phoenixbio\Phoenix Bioinformatics\IF Code\cleanedDatasets\phyb.csv')
  #df = df['abstract']
  # Clean Paragraphs
  col_list =  list(main_df["abstract"])

  documents_clean = []
  for d in col_list:
    if isinstance(d, str):
        document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
        document_test = re.sub(r'@\w+', '', document_test)
        document_test = document_test.lower()
        document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
        document_test = re.sub(r'[0-9]', '', document_test)
        document_test = re.sub(r'\s{2,}', ' ', document_test)
        documents_clean.append(document_test)

  return documents_clean, main_df

In [11]:
docs,main_df = retrieve_docs_and_clean()

# Create Term-Document Matrix with TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)



In [12]:
# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
print(df.head())

            0    1         2    3    4    5    6         7    8    9   ...  \
aba        0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...   
ability    0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.070121  0.0  0.0  ...   
abiotic    0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...   
abolishes  0.0  0.0  0.058307  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...   
aborted    0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...   

            43   44   45   46        47        48   49        50   51   52  
aba        0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0  
ability    0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.052743  0.0  0.0  
abiotic    0.0  0.0  0.0  0.0  0.079697  0.285453  0.0  0.000000  0.0  0.0  
abolishes  0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0  
aborted    0.0  0.0  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0  

[5 rows x 53 columns]


In [13]:
def get_similar_articles(q, df):
  print("query:", q)
  print("The following are articles with the highest cosine similarity values: ")
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  for i in df.columns:
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  
  for k, v in sim_sorted:
    if v != 0.0:
      #print("Similarity Value:", v)
      print(docs[k])
      print(k)
      print(main_df_temp[k])
      return


q1 = 'photoreceptor active in photomorphogenesis phyB'
main_df_temp=list(main_df['pmcid'])
get_similar_articles(q1, df)
print('-'*100)

query: photoreceptor active in photomorphogenesis phyB
The following are articles with the highest cosine similarity values: 
photomorphogenesis refers to photoreceptor mediated morphological changes in plant development that are triggered by light multiple photoreceptors and transcription factors tfs are involved in the molecular regulation of photomorphogenesis likewise light can also modulate the outcome of plant virus interactions since both photosynthesis and many viral infection events occur in the chloroplast despite the apparent association between photosynthesis and virus infection little is known about whether there are also interplays between photomorphogenesis and plant virus resistance recent research suggests that plant virus interactions are potentially regulated by several photoreceptors and photomorphogenesis regulators including phytochromes a and b phya and phyb cryptochromes cry phototropin phot the photomorphogenesis repressor constitutive photomorphogenesis cop th

In [14]:
import metapub
from urllib.request import urlretrieve
import textract

pmid = '28743949'

url = metapub.FindIt(pmid).url

urlretrieve(url, r"C:\Users\ammar\OneDrive\Desktop\phoenixbio\Phoenix Bioinformatics\any")

with open(r'C:\Users\ammar\OneDrive\Desktop\phoenixbio\Phoenix Bioinformatics/l', "w") as textfile:
    textfile.write(textract.process(
        r"C:\Users\ammar\OneDrive\Desktop\phoenixbio\Phoenix Bioinformatics\any",
        extension='pdf',
        method='pdftotext',
        encoding="utf_8",
    ))

2023-03-04 09:37:06 DESKTOP-5C4MFAB metapub.config[22532] WARNING NCBI_API_KEY was not set.
2023-03-04 09:37:06 DESKTOP-5C4MFAB metapub.findit[22532] INFO FindIt Cache initialized at C:\Users\ammar\.cache\findit.db


TypeError: write() argument must be str, not bytes